# Loads Top 100 NASDAQ daily stock prices DATASET from Kaggle + yfinance DATASET


In [ ]:
# import shutil
# import os 

# # Delete the entire yfinance folder and its contents
# shutil.rmtree("../data/raw/yfinance", ignore_errors=True)

# # Remove Kaggle CSV files but keep the folder
# for file in os.listdir("../data/raw"):
#     if file.endswith(".csv"):
#         os.remove(f"../data/raw/{file}")

# print("Old CSV files deleted.")

import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import yfinance as yf

# Authenticate Kaggle API
api = KaggleApi()
api.authenticate()

# Download Kaggle dataset
dataset = "svaningelgem/nasdaq-100-daily-stock-prices"
api.dataset_download_files(dataset, path="../data/raw", unzip=True)

# Get the first CSV file from Kaggle dataset
downloaded_files = os.listdir("../data/raw")
csv_file = [f for f in downloaded_files if f.endswith('.csv')][0]

# Load Kaggle dataset
df_kaggle_data = pd.read_csv(f"../data/raw/{csv_file}")
print(f"Loaded Kaggle dataset with {len(df_kaggle_data)} rows")
print(df_kaggle_data.head(5))
print(df_kaggle_data.tail(5))

# Download stock prices from Yahoo Finance
tickers = ["AAPL", "MSFT", "GOOGL"]
for ticker in tickers:
    df_yfinance = yf.download(ticker, period="1y")  # Store yfinance data in a separate variable
    df_yfinance.to_csv(f"../data/raw/yfinance/{ticker}.csv") 
    print(f"Saved {ticker} data to ../data/raw/yfinance/{ticker}.csv")

# Print a summary
print("\nKaggle files:", os.listdir("../data/raw")[:3])  
print("Yfinance files:", os.listdir("../data/raw/yfinance"))


Old CSV files deleted.
